# Download Data

Imports

In [ ]:
!pip install -q baostock -i https://pypi.tuna.tsinghua.edu.cn/simple/ --trusted-host pypi.tuna.tsinghua.edu.cn

In [2]:
import baostock as bs
import pandas as pd
from tqdm import tqdm

## Index composition of CSI500 index at date = '2021-01-01'

In [ ]:
# login system
lg = bs.login()
# Display login return information
print('login respond error_code:'+lg.error_code)
print('login respond error_msg:'+lg.error_msg)

# Get CSI 500 constituent stocks
rs = bs.query_zz500_stocks(date = '2021-01-01')
print('query_zz500 error_code:'+rs.error_code)
print('query_zz500 error_msg:'+rs.error_msg)

#Print the result set
zz500_stocks = []

with tqdm(total=500, unit="record") as pbar:
    while rs.error_code == '0' and rs.next():
        # Get a record and merge the records together
        zz500_stocks.append(rs.get_row_data())
        pbar.update(1)

stock_composition = pd.DataFrame(zz500_stocks, columns=rs.fields)
# Output the result set to a csv file
stock_composition.to_csv("./CSI Index Data/zz500_stocks_2021-01-01.csv", encoding="utf-8", index=False)

# Log out of the system
bs.logout()

In [3]:
file_path = './CSI Index Data/zz500_stocks_2021-01-01.csv'
stock_composition = pd.read_csv(file_path)
stock_composition.describe()

,updateDate,code,code_name
count,500,500,500
unique,1,500,500
top,2020-12-28,sz.000563,睿创微纳
freq,500,1,1


## 30min bar data from 2022-04-01 to 2022-07-31 for all 500 stocks of the CSI500 index

In [3]:
start_date = '2022-03-15'  #Taking a start date 15 days earlier to be able to calculate features to start trading on 2022-04-01  
end_date = '2022-07-31' 

### Find number of trading days during the given duration

In [4]:
#### login system####
lg = bs.login()
# Display login return information
print('login respond error_code:'+lg.error_code)
print('login respond error_msg:'+lg.error_msg)

#### Get trading day information####
rs = bs.query_trade_dates(start_date=start_date, end_date=end_date)
print('query_trade_dates respond error_code:'+rs.error_code)
print('query_trade_dates respond error_msg:'+rs.error_msg)

#### Print result set####
data_list = []
while (rs.error_code == '0') & rs.next():
    # Get a record and merge the records together
    data_list.append(rs.get_row_data())
trading_days = pd.DataFrame(data_list, columns=rs.fields)

trading_days.to_csv("./CSI Index Data/trading_days.csv", index=False)

#### Log out of the system####
bs.logout()

login success!
login respond error_code:0
login respond error_msg:success
query_trade_dates respond error_code:0
query_trade_dates respond error_msg:success
logout success!


In [4]:
file_path = "./CSI Index Data/trading_days.csv"
trading_days = pd.read_csv(file_path)
print("Trading Days:",trading_days['is_trading_day'].astype(int).sum())

Trading Days: 93


### Get Stock Data

In [7]:
def get_stock_data(stock_code, start_date, end_date):
    
    # Setting adjustflag to be 2 to adjust for dividends, bonuses, and rights issues
    rs = bs.query_history_k_data_plus(stock_code,
        "date,time,code,open,high,low,close,volume,amount,adjustflag",
        start_date=start_date, end_date=end_date,
        frequency="30", adjustflag="2")
    # print('query_history_k_data_plus respond error_code:'+rs.error_code)
    # print('query_history_k_data_plus respond error_msg:'+rs.error_msg)

    data_list = []
    while (rs.error_code == '0') & rs.next():
        # Get a record and merge the records together
        data_list.append(rs.get_row_data())
    
    result = pd.DataFrame(data_list, columns=rs.fields)
    return result

In [8]:
#### login system####
lg = bs.login()
# Display login return information

stock_data_consolidated = pd.DataFrame()

for stock_code in tqdm(stock_composition['code']):
    # print(stock_code)
    stock_data = get_stock_data(stock_code, start_date, end_date)
    stock_data_consolidated = stock_data_consolidated.append(stock_data)

#### Output the result set to a csv file####   
stock_data_consolidated.to_csv("./CSI Index Data/stock_data.csv", index=False)

#### Log out of the system####
bs.logout()

  0%|          | 0/500 [00:00<?, ?it/s]

login success!


100%|██████████| 500/500 [06:31<00:00,  1.28it/s]


logout success!


In [5]:
file_path = "./CSI Index Data/stock_data.csv"
stock_data_consolidated = pd.read_csv(file_path)
display(stock_data_consolidated.tail())

,date,time,code,open,high,low,close,volume,amount,adjustflag
370979,2022-07-29,20220729113000000,sz.300699,43.361023,43.576780,43.151430,43.194581,553807,38977724.0,2
370980,2022-07-29,20220729133000000,sz.300699,43.169923,43.169923,42.719916,42.904850,432687,30084125.0,2
370981,2022-07-29,20220729140000000,sz.300699,42.966495,43.126772,42.904850,42.997318,243000,16955343.0,2
370982,2022-07-29,20220729143000000,sz.300699,42.984989,43.693905,42.941837,43.693905,751600,52897905.0,2
370983,2022-07-29,20220729150000000,sz.300699,43.687741,43.693905,43.114443,43.182252,823460,57858656.0,2


In [9]:
code_counts = stock_data_consolidated['code'].value_counts()
all_days = set(stock_data_consolidated['date'].unique())

# Filter the codes that have less than 744 entries
codes_with_less_than_744_entries = code_counts[code_counts < 744]
# print(codes_with_less_than_744_entries)
days_with_missing_data = set()
days_with_missing_data_dict = {}

for stock_code in codes_with_less_than_744_entries.index:
    stock_data = stock_data_consolidated[stock_data_consolidated['code'] == stock_code]
    missing_days = all_days - set(stock_data['date'])
    # print(missing_days)
    days_with_missing_data_dict[stock_code] = list(missing_days)
    days_with_missing_data = days_with_missing_data.union(missing_days)
print(days_with_missing_data)

{'2022-04-25', '2022-06-21', '2022-04-18', '2022-04-06', '2022-05-27', '2022-06-14', '2022-07-01', '2022-06-29', '2022-06-30', '2022-04-21', '2022-07-04', '2022-07-19', '2022-06-27', '2022-07-25', '2022-07-28', '2022-04-19', '2022-07-08', '2022-05-10', '2022-07-06', '2022-07-26', '2022-06-16', '2022-04-28', '2022-04-01', '2022-06-08', '2022-07-20', '2022-07-22', '2022-04-20', '2022-06-10', '2022-05-31', '2022-03-29', '2022-05-30', '2022-05-16', '2022-06-28', '2022-05-12', '2022-05-11', '2022-05-05', '2022-06-24', '2022-04-27', '2022-07-14', '2022-07-11', '2022-07-15', '2022-06-20', '2022-06-09', '2022-06-01', '2022-07-18', '2022-06-17', '2022-06-02', '2022-06-07', '2022-06-22', '2022-05-09', '2022-07-29', '2022-07-07', '2022-03-30', '2022-06-13', '2022-07-13', '2022-04-29', '2022-06-06', '2022-07-05', '2022-06-23', '2022-05-13', '2022-06-15', '2022-04-26', '2022-07-27', '2022-07-12', '2022-03-31', '2022-05-18', '2022-07-21', '2022-04-22', '2022-05-26', '2022-05-06', '2022-05-25'}


# Benchmark Performance

Remove Stocks that have missing data

In [10]:
# Ommiting the 14 stocks for which 30 min bar data is not available, that leaves us with 486 stocks
stock_data_consolidated = stock_data_consolidated[~stock_data_consolidated['code'].isin(days_with_missing_data_dict.keys())].reset_index(drop=True)
stock_data_consolidated.describe(include='all')

stock_data_consolidated['date'].unique()
print(stock_data_consolidated.shape)

(361584, 10)


In [11]:
# Assuming the availaible amount to be CNY 100M
AMOUNT = 100000000 

def calculate_investment_profit(stock_data=stock_data_consolidated, initial_amount=AMOUNT, start_time=20220401100000000, end_time=20220630150000000):
    if start_time not in stock_data['time'].values:
        raise ValueError("Start time not found in the DataFrame.")
    if end_time not in stock_data['time'].values:
        raise ValueError("End time not found in the DataFrame.")

    n_stocks = stock_data['code'].unique().shape[0]
    per_stock_amount = initial_amount / n_stocks

    stock_prices_start = stock_data.loc[stock_data['time'] == start_time, ['code', 'close']].reset_index(drop=True)
    quantities_bought = per_stock_amount // stock_prices_start['close']
    money_invested_per_stock = stock_prices_start['close'] * quantities_bought
    money_invested = money_invested_per_stock.sum()
    money_left = initial_amount - money_invested

    stock_prices_end = stock_data.loc[stock_data['time'] == end_time, ['code', 'close']].reset_index(drop=True)
    quantities_sold = quantities_bought
    money_retrieved = (stock_prices_end['close'] * quantities_sold).sum()
    closing_amount = money_retrieved + money_left
    profit_pct_based_initial_amount = (closing_amount / initial_amount - 1) * 100

    result = {
        "n_stocks": n_stocks,
        "per_stock_amount": per_stock_amount,
        "money_left": money_left,
        "money_invested": money_invested,
        "money_retrieved": money_retrieved,
        "closing_amount": closing_amount,
        "profit_pct_based_initial_amount": profit_pct_based_initial_amount,
    }

    return result

# Usage example:
# result = calculate_investment_profit(stock_data_consolidated, start_time=20220401100000000, end_time=20220630150000000)
# print(result)


result = calculate_investment_profit()
print(result)

result_out_sample = calculate_investment_profit(start_time=20220701100000000, end_time=20220729150000000)
print(result_out_sample)


{'n_stocks': 486, 'per_stock_amount': 205761.316872428, 'money_left': 4760.6345786601305, 'money_invested': 99995239.36542134, 'money_retrieved': 103202192.18878993, 'closing_amount': 103206952.8233686, 'profit_pct_based_initial_amount': 3.206952823368603}
{'n_stocks': 486, 'per_stock_amount': 205761.316872428, 'money_left': 4725.580471023917, 'money_invested': 99995274.41952898, 'money_retrieved': 98590602.15682344, 'closing_amount': 98595327.73729447, 'profit_pct_based_initial_amount': -1.4046722627055308}


# The Strategy

## Features

In [9]:
# Features
# 1. SMA 
# 2. EMA
# 3. Z-Score
# 4. RSI

def calculate_sma(series , window):
    return series.rolling(window=window).mean()

def calculate_ema(series, span):
    return series.ewm(span=span, adjust=False).mean()

def calculate_zscore(series, window):
    rolling_mean = series.rolling(window=window).mean()
    rolling_std = series.rolling(window=window).std()
    z_score = (series - rolling_mean) / rolling_std
    return z_score

def calculate_rsi(series, window):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

## Backtest

In [11]:
def prepare_indicators(df, long_window, long_to_short_ratio, z_score_centre, long_ma_type, short_ma_type):
    
    short_window = long_window // long_to_short_ratio
    grouped = df.groupby('code')

    if long_ma_type == 'sma':
        long_ma_func = lambda x: calculate_sma(x, long_window)
    elif long_ma_type == 'ema':
        long_ma_func = lambda x: calculate_ema(x, long_window)
    else:
        raise ValueError("Invalid 'long_ma_type'. Use 'sma' or 'ema'.")

    if short_ma_type == 'sma':
        short_ma_func = lambda x: calculate_sma(x, short_window)
    elif short_ma_type == 'ema':
        short_ma_func = lambda x: calculate_ema(x, short_window)
    else:
        raise ValueError("Invalid 'short_ma_type'. Use 'sma' or 'ema'.")

    # Calculate selected moving averages, Z-Score, RSI for each group
    df['long MA'] = grouped['close'].apply(long_ma_func)
    df['short MA'] = grouped['close'].apply(short_ma_func)
    df['Z-Score'] = grouped['close'].apply(lambda x: calculate_zscore(x, long_window))
    df['RSI'] = grouped['close'].apply(lambda x: calculate_rsi(x, short_window))

    return df   

def prepare_filters(df, rsi_l, rsi_u, z_score_centre):
    df['MA_Down_Cross'] = (df['short MA'] < df['long MA']).astype(int)
    df['RSI_b/w_bounds'] = ((df['RSI'] >= rsi_l) & (df['RSI'] <= rsi_u)).astype(int)

    df['Z_Score_Trigger'] = 0
    z_score = df['Z-Score']

    # Apply the logic to set the trigger values
    df.loc[(z_score < 0) & (z_score >= -z_score_centre), 'Z_Score_Trigger'] = 1
    df.loc[(z_score < -z_score_centre) & (z_score >= -2*z_score_centre), 'Z_Score_Trigger'] = 2

    df['weights'] = df['MA_Down_Cross'] * df['RSI_b/w_bounds'] * df['Z_Score_Trigger']
    df['weights_normalized'] = df.groupby('time')['weights'].transform(lambda x: x / x.sum())
    df['weights_normalized'].fillna(0, inplace=True)    
    

    return df 

def trade(df, start_date, end_date, initial_amount):
    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    time_stamps = df['time'].unique()

    portfolio = pd.DataFrame()
    portfolio['stocks'] = df['code'].unique()
    portfolio['quantities'] = 0

    trade_logs = pd.DataFrame(index = time_stamps)
    cash = initial_amount

    for ts in time_stamps:
        
        # Get the rows of the df for current timestamp
        current_timestamp = df[df['time']==ts]
        current_prices = current_timestamp['close'].reset_index(drop = True)
        
        # Find effective money availaible
        stock = (portfolio['quantities'] * current_prices).sum()
        effective_money_available = stock + cash
        
        # Get the ideal portfolio composition for current timestamp
        ideal_weights = current_timestamp['weights_normalized'].reset_index(drop = True)
        per_stock_amount = ideal_weights * effective_money_available
        ideal_composition = per_stock_amount//current_prices

        # Get the changes in composition for current timestamp
        changes_in_composition = ideal_composition - portfolio['quantities']

        # Update new quantities of the stocks in the portfolio and cash availaible
        portfolio['quantities'] = ideal_composition
        money_invested = (ideal_composition * current_prices).sum()
        cash = effective_money_available - money_invested
        
        # Update the trade logs
        trade_logs.loc[ts,'trxns']      = str(list(changes_in_composition))
        trade_logs.loc[ts,'stock']      = money_invested
        trade_logs.loc[ts,'cash']       = cash
        trade_logs.loc[ts,'eff_mon']    = effective_money_available
    
    return trade_logs

# Calls all three functions above
def run_trading_strategy(df, start_date, end_date, long_window, long_to_short_ratio, rsi_bounds, z_score_centre, long_ma_type, short_ma_type, initial_amount):

    df_with_indicators = prepare_indicators(df, long_window, long_to_short_ratio, z_score_centre, long_ma_type, short_ma_type)
    df_with_filters = prepare_filters(df_with_indicators, rsi_bounds[0], rsi_bounds[1], z_score_centre)
    trade_logs = trade(df, start_date, end_date, initial_amount)
    return trade_logs

Backtest for various hyperparam configs

In [12]:
from tqdm import tqdm
import json
# Define the parameter combinations
dates_list = [('2022-04-01', '2022-06-30'), ('2022-07-01', '2022-07-29')]
long_ma_types = ['sma', 'ema']
short_ma_types = ['sma', 'ema']
long_windows = [60, 72, 84, 96]
long_to_short_ratios = [2, 3, 4]
rsi_bounds_list = [(20, 80), (30, 70), (40, 60)]
z_score_centres = [1, 0.75]

# Initialize the progress bar
total_iterations = (
    len(dates_list)
    * len(long_ma_types)
    * len(short_ma_types)
    * len(long_windows)
    * len(long_to_short_ratios)
    * len(rsi_bounds_list)
    * len(z_score_centres)
)
pbar = tqdm(total=total_iterations)
results = []
index = 0
# Iterate through parameter combinations
for dates in dates_list:
    for long_ma_type in long_ma_types:
        for short_ma_type in short_ma_types:
            for long_window in long_windows:
                for long_to_short_ratio in long_to_short_ratios:
                    for rsi_bounds in rsi_bounds_list:
                        for z_score_centre in z_score_centres:
                            trade_logs = run_trading_strategy(
                                df=stock_data_consolidated,
                                start_date=dates[0],
                                end_date=dates[1],
                                long_window=long_window,
                                long_to_short_ratio=long_to_short_ratio,
                                rsi_bounds=rsi_bounds,
                                z_score_centre=z_score_centre,
                                long_ma_type=long_ma_type,
                                short_ma_type=short_ma_type,
                                initial_amount=100000000
                            )

                            # Save configuration and trade logs
                            config = {
                                "dates": dates,
                                "long_ma_type": long_ma_type,
                                "short_ma_type": short_ma_type,
                                "long_window": long_window,
                                "long_to_short_ratio": long_to_short_ratio,
                                "rsi_bounds": rsi_bounds,
                                "z_score_centre": z_score_centre,
                            }
                            results.append({"config": config, "trade_logs": trade_logs})

                            with open(f"./JSONs/config_{index}.json", 'w') as config_file:
                                json.dump(config, config_file)

                            # Save the trade logs as a CSV file
                            trade_logs.to_csv(f"./CSVs/trade_logs_{index}.csv", index=False)

                            # Update the progress bar
                            pbar.update(1)
                            index+=1

# Close the progress bar
pbar.close()


100%|██████████| 576/576 [30:03<00:00,  3.13s/it]


Calculate performance metrics for each hyperparam config backtest

In [41]:
import numpy as np
def calculate_metrics(value_series):
    
    total_return_percent = (value_series.iloc[-1]/value_series.iloc[0]-1) * 100

    cummax = value_series.cummax()
    drawdown = (value_series - cummax) / cummax * 100
    max_drawdown = drawdown.min()

    return_series = value_series.pct_change()
    volatility = return_series.std() * np.sqrt(return_series.size) * 100

    result_dict = {
    'total_return_percent': total_return_percent,
    'volatility': volatility,
    'max_drawdown': max_drawdown,
    'return_per_unit_risk':total_return_percent/volatility
    }

    return result_dict


In [56]:
final_output = []

for index in tqdm(range(total_iterations)):
    # Load the config from JSON
    with open(f"./JSONs/config_{index}.json", 'r') as config_file:
        config = json.load(config_file)

    # Read the trade logs from CSV
    trade_logs = pd.read_csv(f"./CSVs/trade_logs_{index}.csv")
    performance_metrics = calculate_metrics(trade_logs['eff_mon'])
    combined_data = {**config, **performance_metrics}
    final_output.append(combined_data)

result_df = pd.DataFrame(final_output)
result_df.to_csv(f"result_df.csv")

  0%|          | 0/576 [00:00<?, ?it/s]

100%|██████████| 576/576 [00:09<00:00, 63.42it/s] 
